In [1]:
facility = 'room bed comfortable bathroom bar shower view facility wifi floor window air door desk tv decor bedroom'
service = 'staff friendly service reception check booking'
transportation = 'location station area place metro central city parking park'
food = 'breakfast restaurant food coffee water tea drink'
cl = [facility,service,transportation,food]
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
import math
import numpy as np
import pandas as pd
data = pd.read_csv('clean_dataset.csv')
lei = data[data['TripStyle'].isin(['Leisure'])]
bus = data[data['TripStyle'].isin(['Business'])]
del data['Unnamed: 0']

In [2]:
r = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+' #要去除的符號
wnl = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))#載入stopwords
df = {}
tfN = []
tfP = []
for i in range(len(data)):
    nopunc = re.sub(r, "", data['NegativeReview'][i])  # 去除符號
    low = nopunc.lower()  # 轉成小寫
    tokenize = low.split()
    stem = [wnl.lemmatize(i) for i in tokenize]
    content = [r for r in stem if r not in stop_words]  # 去除stopwords
    c = Counter(content)
    tfN.append(c)
    for i2 in c:
        df[i2]=df.get(i2,0)+1
    nopunc = re.sub(r, "", data['PositiveReview'][i])  # 去除符號
    low = nopunc.lower()  # 轉成小寫
    tokenize = low.split()
    stem = [wnl.lemmatize(i) for i in tokenize]
    content = [r for r in stem if r not in stop_words]  # 去除stopwords
    c = Counter(content)
    tfP.append(c)
    for i2 in c:
        df[i2]=df.get(i2,0)+1

In [3]:
V = {}
for k, v in sorted(df.items(), key=lambda q: q[1], reverse = True):
    if v >= 3000:
        V[k] = v
print(V)

{'room': 97588, 'wa': 86314, 'staff': 68315, 'location': 61743, 'hotel': 49027, 'breakfast': 39907, 'negative': 39691, 'good': 36367, 'great': 30867, 'bed': 28213, 'friendly': 26762, 'helpful': 22214, 'nice': 21615, 'clean': 21529, 'small': 20763, 'comfortable': 19665, 'excellent': 18013, 'service': 14774, 'positive': 14191, 'bathroom': 13767, 'nothing': 13644, 'stay': 13641, 'would': 13044, 'really': 12130, 'bar': 11818, 'could': 11778, 'restaurant': 11224, 'one': 10883, 'night': 10820, 'station': 10226, 'everything': 10081, 'close': 10056, 'shower': 10011, 'time': 9966, 'lovely': 9942, 'like': 9920, 'well': 9750, 'reception': 9704, 'bit': 9351, 'also': 8893, 'price': 8735, 'view': 8702, 'little': 8682, 'london': 8606, 'area': 8400, 'food': 8356, 'get': 8151, 'facility': 8093, 'day': 7795, 'u': 7782, 'perfect': 7762, 'quiet': 7637, 'need': 7190, 'check': 7054, 'wifi': 7019, 'even': 6901, 'floor': 6634, 'comfy': 6490, 'walk': 6449, 'place': 6363, 'work': 6349, 'free': 6294, 'next': 614

In [4]:
lenV = len(V)
for i in range(len(tfN)):
    temp = np.zeros(lenV)
    count = 0
    for v in V:
        num = tfN[i].get(v,0)
        if num > 0:
            temp[count]=num
        count = count + 1
    tfN[i] = temp
    
    temp = np.zeros(lenV)
    count = 0
    for v in V:
        num = tfP[i].get(v,0)
        if num > 0:
            temp[count]=num
        count = count + 1
    tfP[i] = temp

In [5]:
cluster = []
for i in cl:
    temp = np.zeros(lenV)
    tokenize = i.split()
    c = Counter(tokenize)
    count = 0
    for v in V:
        num = c.get(v,0)
        if num > 0:
            temp[count]=num
        count = count + 1
    cluster.append(temp)

In [6]:
idf = np.zeros(lenV)
count = 0
N = len(data)*2
for k, v in V.items():#計算idf
    idf[count] = math.log(N/v, 10)
    count = count + 1
for i in range(len(tfN)):
    x = tfN[i] * idf
    x_norm = np.linalg.norm(x)
    tfN[i] = x/x_norm
for i in range(len(tfP)):
    x = tfP[i] * idf
    x_norm = np.linalg.norm(x)
    tfP[i] = x/x_norm

D:\Anaconda3\envs\IR\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
D:\Anaconda3\envs\IR\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [7]:
for i in range(len(cluster)):
    x = cluster[i] * idf
    x_norm = np.linalg.norm(x)
    cluster[i] = x/x_norm

In [8]:
neg_b = np.zeros(5)
pos_b = np.zeros(5)
neg_l = np.zeros(5)
pos_l = np.zeros(5)
for i in range(len(data)):
    similarity = []
    other = 1
    for i2 in range(len(cluster)):
        ans = np.dot(cluster[i2],tfN[i])
        if ans > 0:
            other = 0
        similarity.append(ans)
    if other == 1:
        if data['TripStyle'][i] == 'Business':
            neg_b[4] = neg_b[4] + 1
        else:
            neg_l[4] = neg_l[4] + 1
    else:
        if data['TripStyle'][i] == 'Business':
            index = np.argmax(similarity)
            neg_b[index] = neg_b[index] + 1
        else:
            index = np.argmax(similarity)
            neg_l[index] = neg_l[index] + 1

for i in range(len(data)):
    similarity = []
    other = 1
    for i2 in range(len(cluster)):
        ans = np.dot(cluster[i2],tfP[i])
        if ans > 0:
            other = 0
        similarity.append(ans)
    if other == 1:
        if data['TripStyle'][i] == 'Business':
            pos_b[4] = pos_b[4] + 1
        else:
            pos_l[4] = pos_l[4] + 1
    else:
        if data['TripStyle'][i] == 'Business':
            index = np.argmax(similarity)
            pos_b[index] = pos_b[index] + 1
        else:
            index = np.argmax(similarity)
            pos_l[index] = pos_l[index] + 1

In [9]:
print(neg_b)
print(pos_b)

[26821.  8144.  4278.  9740. 33956.]
[16846. 18810. 19565.  9630. 18088.]


In [10]:
print(neg_l)
print(pos_l)

[24772.  6797.  3652. 10216. 37502.]
[19876. 22219. 18381.  9547. 12916.]


In [11]:
# 1342
# 3214

# 1342
# 2134